In [3]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def make_env():
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

    Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
    in Gymnasium. I parametri usati sono:
    - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
    - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

    Ritorna:
        gym.Env: l'ambiente Ant-v5 inizializzato.
    """
    
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=0.2282706739101626, # scala del rumore quando l'ambiente viene resettato 
                    forward_reward_weight=0.09314040045482441, # peso del reward per il movimento in avanti
                    ctrl_cost_weight=0.028140178122103423, # peso del reward per il controllo
                    healthy_reward =0.9926479631637423, # reward per la salute
                    render_mode='None')

In [5]:
# 1. Creiamo un ambiente vettorializzato (Vectorized Environment)
# Utilizziamo DummyVecEnv per gestire più istanze dell'ambiente come se fossero una singola entità.
# Qui passiamo la funzione make_env (definita in un'altra cella) che crea l'ambiente "Ant-v5".
env = DummyVecEnv([make_env])  

# 2. Normalizziamo osservazioni (obs) e ricompense (reward)
# VecNormalize scala le osservazioni e le ricompense per stabilizzare l'allenamento.
# Parametri:
#   norm_obs=True   -> Abilita la normalizzazione delle osservazioni.
#   norm_reward=True -> Abilita la normalizzazione delle ricompense.
#   clip_obs=10.     -> Limita i valori normalizzati dell'osservazione a un range [-10, 10] per evitare estremi.
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='None' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(


In [6]:
# 3. Definiamo il modello RL (PPO) con spiegazioni dettagliate per ciascun parametro

model = PPO(
    policy="MlpPolicy",           # Tipo di policy: una rete neurale MLP (Multilayer Perceptron) che mappa osservazioni ad azioni
    env=env,                      # Ambiente di addestramento: usa l'ambiente vettorializzato e normalizzato creato in precedenza
    learning_rate=0.0008676828845312949,           # Tasso di apprendimento: controlla la velocità con cui il modello apprende aggiornando i pesi
    n_steps=4096,                 # Numero di passi da eseguire nell'ambiente per ogni ciclo di aggiornamento della policy
    batch_size=64,                # Dimensione del batch per gli aggiornamenti stocastici: suddivide i dati raccolti nei mini-batch
    n_epochs=100,                  # Numero di volte (epoch) che il dataset raccolto viene utilizzato per aggiornare la policy
    gamma=0.9328230070576791,      # Fattore di sconto: determina l'importanza delle ricompense future rispetto a quelle immediate
    gae_lambda=0.95,              # Parametro per il Generalized Advantage Estimation (GAE): bilancia bias e varianza nella stima dell'advantage
    clip_range=0.2,               # Intervallo di clipping: limita le variazioni della policy per mantenere aggiornamenti stabili
    ent_coef=0.0,                 # Coefficiente di entropia: controlla l'incentivo all'esplorazione; 0 significa nessun bonus per l'entropia
    verbose=1,                    # Livello di verbosità: 1 per stampare informazioni di log utili durante l'addestramento
    tensorboard_log="./ppo_Ant_tensorboard/",  # Cartella per salvare i log di TensorBoard
    device='mps'                    # Specifica l'uso della GPU su Apple Silicon
)

Using mps device


/Users/fabiodigregorio/Desktop/campus bio iscrizione/ Magistrale/Merone/RL/Reinforcement_Learning_Ant_MuJoCu/venv/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [7]:
# 4. Alleniamo il modello
# Il parametro total_timesteps indica il numero totale di iterazioni (o passi)
# che il modello eseguirà durante l'allenamento. Ogni timestep rappresenta un'interazione
# con l'ambiente in cui il modello esegue un'azione e riceve un feedback, che viene poi
# usato per aggiornare la sua politica interna.
total_timesteps = 100000  # Puoi aumentare questo valore per permettere al modello di acquisire più esperienza.
model.learn(total_timesteps=total_timesteps)

Logging to ./ppo_Ant_tensorboard/PPO_1
-----------------------------
| time/              |      |
|    fps             | 368  |
|    iterations      | 1    |
|    time_elapsed    | 11   |
|    total_timesteps | 4096 |
-----------------------------


KeyboardInterrupt: 

In [ ]:
#5. Salviamo il modello
model.save("ppo_Ant_model")
env.save("vecnormalize_Ant.pkl")  # salviamo anche i parametri di normalizzazione

In [ ]:
def evaluate_policy(env, policy, episodes=50):
    """
    Valuta una policy addestrata su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - policy: La policy addestrata da valutare.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not done:
            action, _ = policy.predict(obs)  # Predice l'azione da eseguire
            obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense


def evaluate_random_policy(env, episodes=50):
    """
    Valuta una policy casuale su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
        done = False
        total_reward = 0
        while not done:
            action = env.action_space.sample()  # Genera un'azione casuale
            if isinstance(action, np.ndarray):
                action = action.flatten()
            obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
            total_reward += reward  # Accumula la ricompensa ottenuta
        total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
    return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense

In [ ]:
# Valutazione dopo l'addestramento
mean_reward_trained, std_reward_trained = evaluate_policy(env, model)  # Valuta la policy addestrata
mean_reward_random, std_reward_random = evaluate_random_policy(env)  # Valuta la policy casuale

# Stampa dei risultati
print(f"Trained Policy: Mean Reward: {mean_reward_trained}, Std: {std_reward_trained}")
print(f"Random Policy: Mean Reward: {mean_reward_random}, Std: {std_reward_random}")

# Creazione del grafico di confronto
labels = ['Random Policy', 'Trained Policy']
means = [mean_reward_random, mean_reward_trained]
stds = [std_reward_random, std_reward_trained]

plt.figure(figsize=(8, 5))
plt.bar(labels, means, yerr=stds, capsize=10, color=['skyblue', 'lightgreen'])
plt.ylabel('Mean Episodic Reward')
plt.title('Policy Comparison')
plt.show()

In [ ]:
# 6. Valutazione
# Qui creiamo un ambiente specifico per la valutazione del modello
#
# L'obiettivo è osservare in tempo reale come il modello interagisce con l'ambiente.
# Questo ambiente differisce da quello usato durante l'allenamento (che era vettorializzato e normalizzato)
# Per la valutazione, possiamo utilizzare l'ambiente "grezzo" di Gymnasium con rendering.
#
# Parametri:
#   - "Ant-v5": Nome dell'ambiente;
#   - reset_noise_scale=0.1: Scala del rumore applicato durante il reset dell'ambiente,
#       utile per mantenere la coerenza con le condizioni viste durante l'allenamento.
#   - render_mode="human": Abilita il rendering dell'ambiente in tempo reale, permettendoci di osservare visivamente il comportamento del modello.
# eval_env = gym.make("Ant-v5", reset_noise_scale=0.1)
#                     #, render_mode="human")

In [ ]:
# # Ricreiamo il wrapper di normalizzazione con i parametri salvati
# """
# Questo snippet di codice configura un ambiente di valutazione normalizzato per un compito di reinforcement learning.
# Esegue le seguenti operazioni:

# 1. Avvolge l'ambiente originale (eval_env) in un ambiente vettorializzato utilizzando DummyVecEnv.
#     Questo è necessario per la compatibilità con algoritmi che richiedono un'interfaccia vettorializzata.
# 2. Carica i parametri di normalizzazione precedentemente salvati (da 'vecnormalize_Ant.pkl') nel nuovo ambiente.
#     Ciò garantisce che le osservazioni dello stato (e potenzialmente le ricompense) siano normalizzate
#     nello stesso modo in cui sono state normalizzate durante l'addestramento.
# 3. Imposta l'ambiente in modalità di valutazione:
#     - Disabilita la modalità training (eval_env.training = False) in modo che le statistiche di normalizzazione non vengano aggiornate.
#     - Disabilita la normalizzazione della ricompensa (eval_env.norm_reward = False) perché durante la valutazione le ricompense
#       vengono tipicamente utilizzate nella loro forma originale.

# Parametri:
# - eval_env: L'istanza dell'ambiente che deve essere avvolta e normalizzata.
# - "vecnormalize_Ant.pkl": Il percorso del file contenente i parametri di normalizzazione salvati specifici per l'ambiente 'Ant'.
#   Questi parametri regolano la scala delle osservazioni e delle ricompense dell'ambiente.

# Nota:
# Questa configurazione è essenziale durante la valutazione di un modello addestrato per garantire che il preprocessamento
# applicato corrisponda a quello dell'addestramento, senza modificare ulteriormente i parametri di normalizzazione.
# """
# eval_env = DummyVecEnv([lambda: eval_env])
# eval_env = VecNormalize.load("vecnormalize_Ant.pkl", eval_env)
# eval_env.training = False
# eval_env.norm_reward = False  # In valutazione di solito non normalizziamo la ricompensa


In [ ]:
# # Reset dell'ambiente di valutazione per ottenere l'osservazione iniziale.
# obs = eval_env.reset()

# # Inizializzazione della variabile che accumulerà la ricompensa totale ottenuta nell'episodio.
# episode_reward = 0.0

# # Variabile booleana per controllare se l'episodio è terminato o troncato.
# done = False

# # Ciclo per eseguire l'iterazione dell'episodio
# while not done:
#     # Otteniamo l'azione in modalità deterministica dal modello (questo assicura che il modello non agisca con esplorazione casuale)
#     action, _states = model.predict(obs, deterministic=True)
    
#     # Eseguiamo l'azione nell'ambiente di valutazione utilizzando il metodo step per ottenere
#     # - observation: l'osservazione successiva
#     # - reward: la ricompensa ottenuta per l'azione intrapresa
#     # - terminated: flag che indica se l'episodio è terminato normalmente
#     # - truncated: flag che indica se l'episodio è stato interrotto (es. timeout)
#     observation, reward, terminated, truncated = eval_env.step(action)
    
#     # Aggiornamento della ricompensa totale: poiché l'ambiente è vettorializzato, reward è un array
#     episode_reward += reward[0]  
#     print(reward[0])
    
#     # L'episodio termina se risulta terminato (terminated=True) oppure se è troncato (truncated=True)
#     done = terminated or truncated

# # Stampa della ricompensa totale accumulata durante l'episodio.
# print("Ricompensa totale episodio:", episode_reward)

# # Chiusura dell'ambiente per liberare le risorse.
# eval_env.close()

-0.86552584
Ricompensa totale episodio: -0.86552584
